# Text Processing & Labelling [Part1]
> Sample of text preprocessing.

- toc: true 
- badges: false
- comments: true
- categories: [implementation]
- image: images/text_processing.png

The full notebook is availabel [here](https://github.com/minhdang241/minhdg-blog/blob/master/_notebooks/2021-07-27-NLP_7_Text_Preprocessing_and_Labelling_%5BPart_1%5D.ipynb).

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
%%capture
!pip install transformers
!pip install emoji
!pip install Unidecode
!pip install contractions
!pip install word2number
!python -m spacy download en_core_web_md
!pip install spacy-langdetect

# Load Dataset

In [195]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Text Classification/Pipeline/datasets.csv")

In [75]:
df.dropna(inplace=True) # drop row(s) containing "nan" value

For those who want to reproduce the note book, you can get the data from [here](https://drive.google.com/file/d/1PZWX6P5bD5_sCIKKPN9dldYy64eW7MGE/view?usp=sharing).

In [187]:
import re
import string
from itertools import groupby

import contractions
import emoji
import unidecode
from bs4 import BeautifulSoup
from spacy_langdetect import LanguageDetector
from word2number import w2n

In [250]:
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # Set to display full-width dataframe 

# Preprocess data

## Step 1: Remove HTML Tags

HTLM tags does not add any useful information for understanding and analyzing text, so we remove it.

In [77]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text


In [84]:
df["no_html"] = df["review"].apply(remove_html_tags)

In [91]:
df[df["no_html"] != df["review"]][100:120]

,comment_id,review,star_rating,no_html
8897,255542,"The colors are super vibrant &amp; unique. I was unable to find colors like this in multiple stores I visited, let alone find both a left &amp; right controller together so this was definitely a win-win.",5,"The colors are super vibrant & unique. I was unable to find colors like this in multiple stores I visited, let alone find both a left & right controller together so this was definitely a win-win."
8955,255550,"I received my Neon Pink &amp; Neon Green Joy Cons yesterday and I'm amazed at how vibrant and colorful this particular set is! These 2 colors compliment each other perfectly, too! 5/5 Stars! Good packaging, prompt next-day delivery as a Prime Member.",5,"I received my Neon Pink & Neon Green Joy Cons yesterday and I'm amazed at how vibrant and colorful this particular set is! These 2 colors compliment each other perfectly, too! 5/5 Stars! Good packaging, prompt next-day delivery as a Prime Member."
9216,255962,Love my Switch &amp; these are fun colors,5,Love my Switch & these are fun colors
9293,256306,&lt;3,5,<3
9466,255331,Perfect pair of new joycons I'm using for a shell &amp; button mod swap.,5,Perfect pair of new joycons I'm using for a shell & button mod swap.
10566,257919,Works &amp; bright colors!,5,Works & bright colors!
10717,258185,"Although we do enjoy the new features that Switch have over Wii:* Does not require sensor bar* Portable - able to play on-the-go with build-in console batteryThere are few limitations that we were not aware prior to purchase:* Joy Con battery not removable. (Since rechargeable batteries have limited life, customers will need to purchase new controllers every 1-2 years depending on amount of use)* Joy Con controller size are small for adults including the joystick &amp; buttons* Gel-Con Guards need to be removed when switching to Wheel adopter* Hand Strap need to removed in order to place on charger",3,"Although we do enjoy the new features that Switch have over Wii:* Does not require sensor bar* Portable - able to play on-the-go with build-in console batteryThere are few limitations that we were not aware prior to purchase:* Joy Con battery not removable. (Since rechargeable batteries have limited life, customers will need to purchase new controllers every 1-2 years depending on amount of use)* Joy Con controller size are small for adults including the joystick & buttons* Gel-Con Guards need to be removed when switching to Wheel adopter* Hand Strap need to removed in order to place on charger"
10737,258240,"After a few months of use, without dropping them, nothing smeared on them, and just playing Smash Bros Ultimate, one of the sticks on a Joy-con is starting to wear down. I can't make it respond to sudden movements or flicking. My left stick, in particular, is now drifting. I tried calibrating the controls, and it's fine, but it did nothing to fix the issue.I looked up on means to fix this, and it requires the risk of ruining your controller further by taking it apart to replace the stick with a new one (or at least adjust the current one). Even then, it does not take long for the drifting to come back.It's entirely possible I got unlucky and received a bad pair with my Switch, but it doesn't change the fact that something like this occurred so soon for something so expensive. I really want to enjoy my games on-the-go more often than at home, but it's not going to happen the way the design is flawed.Even the Pro-Controller is having the same issue... so now I'm hesitant to buy any Nintendo-licensed controller... and it doesn't help that the warranty for these controllers are so short. Terrific. Thanks for nothing, Nintendo.&gt;.&gt;",2,"After a few months of use, without dropping them, nothing smeared on them, and just playing Smash Bros Ultimate, one of the sticks on a Joy-con is starting to wear down. I can't make it respond to sudden movements or flicking. My left stick, in particular, is now drifting. I tried calibrating 

One notice from the example above is that the BeautifulSoup function convert  `&amp;` to `$`. In general, the function tries to convert decode the **HTML Character Entities**.

## Step 2: Remove redundant elements

In [115]:
# Remove and replace by empty space
def remove_redundant_elements(text):
    # remove urls
    text = re.sub(r"http\S+", " ", text)
    # remove phone
    text = re.sub(r"[\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}", " ", text)
    # remove email
    text = re.sub(r"[\w.+-]+@[\w-]+\.[\w.-]+", " ", text)
    # remove newline
    table = str.maketrans("\n\t\r", "   ")
    text = text.translate(table)
    # remove redundant whitespaces
    text = " ".join(text.split())
#     # remove punctuations
#     table_ = str.maketrans(string.punctuation, " " * len(string.punctuation))
#     comment = " ".join(comment.translate(table_).split())
    return text

In [122]:
df["no_redundant"] = df["no_html"].apply(remove_redundant_elements)

In [123]:
df[df["no_redundant"] != df["no_html"]][["no_html", "no_redundant"]]

no_html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

## Step 3: Remove emoji

In [126]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(r" ", text)

In [132]:
df["no_emoji"] = df["no_redundant"].apply(remove_emoji)

In [133]:
df[df["no_redundant"] != df["no_emoji"]][["no_redundant", "no_emoji"]]

,no_redundant,no_emoji
19,I have been using my renewed computer 👩🏾‍💻 for about 2 weeks now and I am so glad I bought it from “All-out-Apple” the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works!! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too.,I have been using my renewed computer for about 2 weeks now and I am so glad I bought it from “All-out-Apple” the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works!! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too.
239,Great purchase! Worth every penny. And the color is gorgeous 😍,Great purchase! Worth every penny. And the color is gorgeous
363,"I spent a lot of time debating on purchasing brand new or renewed off amazon. I ended up going the cheaper route and went with the amazon renewed one. The product came in a plain brown box, but was wrapped like it was brand new. There are no marks or dents on mine, and it acts like a brand new computer!! I am obsessed with it!!! Best purchase I’ve made all year. Seriously, great bang for your buck! 😁","I spent a lot of time debating on purchasing brand new or renewed off amazon. I ended up going the cheaper route and went with the amazon renewed one. The product came in a plain brown box, but was wrapped like it was brand new. There are no marks or dents on mine, and it acts like a brand new computer!! I am obsessed with it!!! Best purchase I’ve made all year. Seriously, great bang for your buck!"
475,I'm happy with product. Is apple 🍎 there's not much to say. there product are great. I'm a prime member. I received it 2 days later. My daughter needed for school on line,I'm happy with product. Is apple there's not much to say. there product are great. I'm a prime member. I received it 2 days later. My daughter needed for school on line
527,I’m in love with my new Mac book very easy to use and set up. I Definitely recommend it great purchase that I did 👌😁,I’m in love with my new Mac book very easy to use and set up. I Definitely recommend it great purchase that I did
...,...,...
11874,"Arrived broken, you can see in the one pic how it came, when my husband took it out of the plastic he said this isn’t right. The black part was completely unattached to the red main part. Upon inspection, because my husband can fix anything...it looks like someone just ripped it off the main part of the controller. It’s missing one screw and the other was ripped out with the red con plastic still on it. It’s beyond repair 😒. How did it pass inspection?!","Arrived broken, you can see in the one pic how it came, when my husband took it out of the plastic he said this isn’t right. The black part was completely unattached to the red main part. Upon inspection, because my husband can fix anything...it looks like someone just ripped it off the main part of the controller. It’s missing one screw and the other was ripped out with the red con plastic still on it. It’s beyond repair . How did it pass inspection?!"
11894,"Had original gray Joy-Cons when I got my Switch and had the dreaded analog drift. It was time I purchased official Nintendo (pricey) Joy-Cons...and always thought this 

All the emojies detected are removed

## Step 4: Expand Contractions

In [153]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

In [154]:
df["expand_contractions"] = df["no_emoji"].apply(expand_contractions)

In [160]:
df[df["expand_contractions"] != df["no_emoji"]][["no_emoji", "expand_contractions"]]

,no_emoji,expand_contractions
1,"Best computer I’ve ever own!! This computer forces me to be productive. I used to wait around for this spinning wheel to stop, and now I can do everything so quickly that I think I almost miss the waiting part. Ha ha. I don’t have time to get coffee anymore. All my apps run smoothly, including adobe illustrator and Photoshop, and they work perfectly. Almost too perfect. I really love this computer. I hope it last me for many years to come.","Best computer I have ever own!! This computer forces me to be productive. I used to wait around for this spinning wheel to stop, and now I can do everything so quickly that I think I almost miss the waiting part. Ha ha. I do not have time to get coffee anymore. All my apps run smoothly, including adobe illustrator and Photoshop, and they work perfectly. Almost too perfect. I really love this computer. I hope it last me for many years to come."
4,"I left my brand new Macbook pro in my car for an hour and now the screen looks like a Picasso painting. Neither Amazon nor Macintosh will pay for repairs, even though I bought it only two months ago, so I'm out $700 to repair my brand new computer. I would heavily advise against buying one of these, because if 80 degree heat is enough to damage the screen, then they're essentially useless if your air conditioner goes out and you will be left with a repair bill that is half the value of the computer.","I left my brand new Macbook pro in my car for an hour and now the screen looks like a Picasso painting. Neither Amazon nor Macintosh will pay for repairs, even though I bought it only two months ago, so I am out $700 to repair my brand new computer. I would heavily advise against buying one of these, because if 80 degree heat is enough to damage the screen, then they are essentially useless if your air conditioner goes out and you will be left with a repair bill that is half the value of the computer."
8,Product came in excellent condition. Battery life isn’t as long as I would like it to be. Product overall works excellent.,Product came in excellent condition. Battery life is not as long as I would like it to be. Product overall works excellent.
9,"I don’t what to think I bought 2017 16gb 256 ssd but I received a refurbished 2019 8g 256 perfectly packaged, I can’t put apple care I already call for that but im not sure to keep it because I want a computer of 16of ram but this one is 2019, im gonna give the chance to see how it works, And it cost me the same price of a refurbished 2019 Of apple but those ones you can put apple care","I do not what to think I bought 2017 16gb 256 ssd but I received a refurbished 2019 8g 256 perfectly packaged, I cannot put apple care I already call for that but I am not sure to keep it because I want a computer of 16of ram but this one is 2019, I am going to give the chance to see how it works, And it cost me the same price of a refurbished 2019 Of apple but those ones you can put apple care"
15,"KEYBOARD FAILED! I ordered this in November and started using it in January as my main “home” laptop (minimal use). The keyboard stopped working properly in June. Some letters don’t regularly work. Sometimes - but not always- the space bar leaves two spaces instead of one. I wish I spent the money to get a NEW laptop and not refurbished. This was my first refurbished purchase, which I now see is a mistake. I live in a remote area, so fixing a keyboard may outweigh the cost savings. The battery works fine. Regrets here.","KEYBOARD FAILED! I ordered this in November and started using it in January as my main “home” laptop (minimal use). The keyboard stopped working properly in June. Some letters do not regularly work. Sometimes - but not always- the space bar leaves two spaces instead of one. I wish I spent the money to get a NEW laptop and not refurbished. This was my first refurbished purchase, which I now see is a mistake. I live in a remote area, so fixing a keyboard may outweigh the cost sav

## Step 5: Removing accented characters

In [161]:
import unidecode
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

In [162]:
df["no_accented"] = df["expand_contractions"].apply(remove_accented_chars)

In [164]:
df[df["expand_contractions"] != df["no_accented"]][["expand_contractions", "no_accented"]]

,expand_contractions,no_accented
15,"KEYBOARD FAILED! I ordered this in November and started using it in January as my main “home” laptop (minimal use). The keyboard stopped working properly in June. Some letters do not regularly work. Sometimes - but not always- the space bar leaves two spaces instead of one. I wish I spent the money to get a NEW laptop and not refurbished. This was my first refurbished purchase, which I now see is a mistake. I live in a remote area, so fixing a keyboard may outweigh the cost savings. The battery works fine. Regrets here.","KEYBOARD FAILED! I ordered this in November and started using it in January as my main ""home"" laptop (minimal use). The keyboard stopped working properly in June. Some letters do not regularly work. Sometimes - but not always- the space bar leaves two spaces instead of one. I wish I spent the money to get a NEW laptop and not refurbished. This was my first refurbished purchase, which I now see is a mistake. I live in a remote area, so fixing a keyboard may outweigh the cost savings. The battery works fine. Regrets here."
19,I have been using my renewed computer for about 2 weeks now and I am so glad I bought it from “All-out-Apple” the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works!! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too.,"I have been using my renewed computer for about 2 weeks now and I am so glad I bought it from ""All-out-Apple"" the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works!! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too."
24,"Excelente producto!! Sin palabras!, Lo mejor para el trabajo, gaming, día a día, el mejor producto de apple hasta la fecha!!!","Excelente producto!! Sin palabras!, Lo mejor para el trabajo, gaming, dia a dia, el mejor producto de apple hasta la fecha!!!"
36,"Wow, absolutely perfect! I bought this for my daughter’s birthday and for her to use at college. She loves it!! It arrived super fast and in perfect condition!! We are extremely pleased with this MacBook. Thanks for a great purchase!","Wow, absolutely perfect! I bought this for my daughter's birthday and for her to use at college. She loves it!! It arrived super fast and in perfect condition!! We are extremely pleased with this MacBook. Thanks for a great purchase!"
48,"I am kindof dissatisfied. So I just opened it and cannot tell you everything, however I can tell you what I do not like that I already found. The apple emblem is cracked and the power cord that it came with has MANY scratches, bumps, and scrapes all over it so I question if the cable even charges properly. I know this is referbished, but can we please follow the policy ""external finishes like new” legit? Aside from those so far I am satisfied. Scrolling on the mouse pad is Kindof meh but maybe it is because I am not use to it yet? The keys are in fine condition and the finger print scanner works. I was worried with some of the reviews saying that they got a grey instead of a space grey but mine is space grey so I am very happy. My fave color. I can leave another review with more information after a week or so of 

## Step 6: Collapse duplicated punctuation

In [184]:
punc = string.punctuation

def remove_consecutive_punctuation(text):
    newtext = []
    for k, g in groupby(text):
        if k in punc:
            newtext.append(k + " ")
        else:
            newtext.extend(g)

    return ''.join(newtext) 

In [ ]:
remove_consecutive_punctuation("Hello World..Now I can see you")

In [185]:
df["no_consecutive_punc"] = df["expand_contractions"].apply(remove_consecutive_punctuation)

In [186]:
df[df["expand_contractions"] != df["no_consecutive_punc"]][["expand_contractions", "no_consecutive_punc"]]

,expand_contractions,no_consecutive_punc
1,"Best computer I have ever own!! This computer forces me to be productive. I used to wait around for this spinning wheel to stop, and now I can do everything so quickly that I think I almost miss the waiting part. Ha ha. I do not have time to get coffee anymore. All my apps run smoothly, including adobe illustrator and Photoshop, and they work perfectly. Almost too perfect. I really love this computer. I hope it last me for many years to come.","Best computer I have ever own! This computer forces me to be productive. I used to wait around for this spinning wheel to stop, and now I can do everything so quickly that I think I almost miss the waiting part. Ha ha. I do not have time to get coffee anymore. All my apps run smoothly, including adobe illustrator and Photoshop, and they work perfectly. Almost too perfect. I really love this computer. I hope it last me for many years to come."
11,"Positivo, EXCELENTE EQUIPO!!!","Positivo, EXCELENTE EQUIPO!"
19,I have been using my renewed computer for about 2 weeks now and I am so glad I bought it from “All-out-Apple” the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works!! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too.,I have been using my renewed computer for about 2 weeks now and I am so glad I bought it from “All-out-Apple” the best computer seller on Amazon! I received the 2018 MacBook Air i5 quad core 256gb and the actual specs matched the description! I use it for college for my Zoom classes and the virtual background works! I upgraded from my 2013 MacBook Air and this one is a lot faster and a much better battery life. It also came with a brand new usb-c original Apple charger for this Mac. This computer is like-new with no scratches at all and I was able to transfer all my old data to this new computer. I would recommend this computer for students and teachers too.
24,"Excelente producto!! Sin palabras!, Lo mejor para el trabajo, gaming, día a día, el mejor producto de apple hasta la fecha!!!","Excelente producto! Sin palabras!, Lo mejor para el trabajo, gaming, día a día, el mejor producto de apple hasta la fecha!"
26,"I am extremely pleased with this early 2015, 13"" Macbook Pro! There is not one mark on the case, screen, keyboard - it is exactly like the brand-new one I purchased from Apple, 5 years ago!! The battery cycle count is only 21 - truly remarkable! It also came with a genuine MagSafe 2 Apple charger (I did check, and it is new and authentic). I am very happy with the purchase so far - thank you for selling such a quality item!","I am extremely pleased with this early 2015, 13"" Macbook Pro! There is not one mark on the case, screen, keyboard - it is exactly like the brand-new one I purchased from Apple, 5 years ago! The battery cycle count is only 21 - truly remarkable! It also came with a genuine MagSafe 2 Apple charger (I did check, and it is new and authentic). I am very happy with the purchase so far - thank you for selling such a quality item!"
...,...,...
12405,"These were an early Christmas gift for my son who has everything he wants, he literally does not want or need anything, so I am always far stretched when trying to figure out what to give him for birthdays and Christmas.I was relieved as heck when he said he wanted a set of these controllers. I could not wait for Christmas because he had the new Super Mario Party game and we were having a few people come to the house to play it.The controllers arrived just in time for our Mario Party , but my son and I played to 

## Step 7: Filter Languages (English only)

In [252]:
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [253]:
def detect_lang(text):
    doc = nlp(text)
    lang = doc._.language['language']
    return lang

In [254]:
df["lang"] = df["no_consecutive_punc"].apply(detect_lang)

In [255]:
non_english_df = df[df.lang != "en"][["no_consecutive_punc", "lang"]]
print("number of non english comments", non_english_df.shape)

number of non english comments (2559, 2)


In [256]:
df.to_csv("preprocessed.csv")

In [258]:
filtered_df = df[df.lang == "en"]

In [260]:
print("Number of data before filter:", len(df))
print("Number of data after filter:", len(filtered_df))

Number of data before filter: 12416
Number of data after filter: 9857


## Step 8: Lower case

In [ ]:
df['lower_case'] = df["no_consecutive_punc"].str.lower()

In [ ]:
df.to_csv("preprocessed.csv", index=False)

# Discussion

There some other techniques that can be considered depends on your task at hand. For example, remove numbers or convert text to number or vice versa.